In [4]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np
import json
import math

In [2]:
symbol="IBM"
timeInterval = "1d"
timeRange = "10y"

In [5]:
def metadata_price_action(ticker, interval, timeRange):
    stockData = requests.get(f"https://query1.finance.yahoo.com/v8/finance/chart/{ticker}?region=US&lang=en-US&includePrePost=false&interval={interval}&useYfid=true&range={timeRange}&corsDomain=finance.yahoo.com&.tsrc=finance")
    stockJson = stockData.json()
    stockResult = stockJson["chart"]["result"][0]
    stockMetadata = stockResult["meta"]
    stockPriceData = pd.DataFrame({"timestamp": stockResult["timestamp"]})
    stockIndicators = pd.DataFrame(stockResult["indicators"]["quote"][0])
    stockDataMerge = pd.concat([stockPriceData, stockIndicators], axis=1)
    return {"metadata": stockMetadata, "price":stockDataMerge}

In [6]:
 def extended_stock_stats(ticker):
    stockData = requests.get(f"https://finance.yahoo.com/quote/{ticker}/key-statistics?p={ticker}")
    soup = BeautifulSoup(stockData.content, 'html.parser')
    job_elems = soup.find_all('section',  {"data-test":"qsp-statistics"})
    datObj={}
    dat=[]
    for job_elem in job_elems:
        for t in job_elem.find_all("tr"):
            if(len(dat)==2):
                head, *tail=np.array(dat)
                datObj[head]=tail[0]
            elif(len(dat)>=2):
                head, *tail=np.array(dat)
                datObj[head]=tail
            dat=[]
            for td in t.find_all("td"):
                dat.append(td.text)
    return datObj


In [7]:
 def basic_stats(ticker):
    statsData = requests.get(f"https://finance.yahoo.com/quote/{ticker}?p={ticker}&.tsrc=fin-srch")
    statsSoup = BeautifulSoup(statsData.content, 'html.parser')
    stats_elem = statsSoup.find('div',id="quote-summary")
    statsObj={}
    stats=[]
    if stats_elem != None:
        for t in stats_elem.find_all("tr"):
            if(len(stats)==2):
                head, *tail=np.array(stats)
                statsObj[head]=tail[0]
            elif(len(stats)>=2):
                head, *tail=np.array(stats)
                statsObj[head]=tail
            stats=[]
            for td in t.find_all("td"):
                stats.append(td.text)
    return statsObj

In [8]:
def current_sp500_symbols():
    sp500 = requests.get("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
    soup = BeautifulSoup(sp500.content, 'html.parser')
    symbols=[]
    symbol_table = soup.find_all('table', id="constituents")
    for sym in symbol_table:
        for tr in sym.find_all("tr"):
            td = tr.find("td")
            if(td != None):
                symbols.append(td.text.strip())
    return symbols

In [9]:
def parse_value(val):
    lastChar = val[-1]
    number = val[0:-1]
    number = number.replace(",","")
    if (lastChar == "M"):
        return float(number) * 1000000
    elif (lastChar == "B"):
        return float(number) * 1000000000
    elif (lastChar == "T"):
        return float(number) * 1000000000000
    elif (lastChar == "%"):
        return float(number) / 100
    else:
        return float(number)

In [10]:
sp500symbols = np.array(current_sp500_symbols())
sp500symbols

array(['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP',
       'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALXN',
       'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN',
       'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP',
       'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'ANTM', 'AON', 'AOS',
       'APA', 'AAPL', 'AMAT', 'APTV', 'ADM', 'ANET', 'AJG', 'AIZ', 'T',
       'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'BKR', 'BLL', 'BAC',
       'BK', 'BAX', 'BDX', 'BRK.B', 'BBY', 'BIO', 'BIIB', 'BLK', 'BA',
       'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BF.B', 'CHRW',
       'COG', 'CDNS', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT',
       'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CERN', 'CF',
       'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS',
       'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'KO', 'CTSH',
       'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'COO', 'CPRT',


In [11]:
my_columns = ["Ticker", "Stock Price", "Market Cap", "52 Week Change", "Number of Shares to Buy"]
dataframe = pd.DataFrame(columns=my_columns)
for sym in sp500symbols[0:4]:
    if(sym.find(".") > -1):
        sym = sym.replace(".", "-")
    basic = basic_stats(sym)
    complexStats = extended_stock_stats(sym)
    marketCap = parse_value(basic["Market Cap"])
    previousClose = parse_value(basic["Previous Close"])
    fiftyTwoWeekChange = previousClose / (1+parse_value(complexStats["52-Week Change 3"]))
    dataframe = dataframe.append(pd.Series([sym,previousClose,round(marketCap),fiftyTwoWeekChange,"N/A"], index=my_columns), ignore_index=True)
    print(sym, round(marketCap), previousClose)
dataframe.to_csv("lastDataframe_bak.csv")
dataframe

MMM 102286000000 178.7
ABT 226898000000 128.2
ABBV 183962000000 104.4
ABMD 14600000000 328.3


,Ticker,Stock Price,Market Cap,52 Week Change,Number of Shares to Buy
0,MMM,178.7,102286000000,158.886814,N/A
1,ABT,128.2,226898000000,88.861163,N/A
2,ABBV,104.4,183962000000,93.573541,N/A
3,ABMD,328.3,14600000000,168.965517,N/A


In [54]:
dataframe.to_csv("lastDataframe.csv")

In [51]:
portfolio_size = input("How large is your portfolio?")

How large is your portfolio? 1000000


In [52]:
portfolio_size = float(portfolio_size)
portfolio_size

1000000.0

In [55]:
position_size = portfolio_size / len(dataframe.index)
print(position_size)
for i in range(0,len(dataframe.index)):
    dataframe.loc[i,"Number of Shares to Buy"] = math.floor(position_size/dataframe.loc[i,"Stock Price"])

dataframe.to_csv("lastDataframeWithSharesToBuy.csv")
dataframe

1980.1980198019803


,Ticker,Stock Price,Market Cap,Number of Shares to Buy
0,MMM,177.2,103485000000,11
1,ABT,126.8,227270000000,15
2,ABBV,103.7,184386000000,19
3,ABMD,322.9,14852000000,6
4,ACN,258.9,170590000000,7
...,...,...,...,...
500,YUM,105.4,31641000000,18
501,ZBRA,470.0,25544000000,4
502,ZBH,162.2,33451000000,12
503,ZION,48.9,8136000000,40


In [8]:
metadataAndPrice = metadata_price_action(symbol, timeInterval, timeRange)
metadata = metadataAndPrice["metadata"]
metadata

{'currency': 'USD',
 'symbol': 'IBM',
 'exchangeName': 'NYQ',
 'instrumentType': 'EQUITY',
 'firstTradeDate': -252322200,
 'regularMarketTime': 1613163602,
 'gmtoffset': -18000,
 'timezone': 'EST',
 'exchangeTimezoneName': 'America/New_York',
 'regularMarketPrice': 120.8,
 'chartPreviousClose': 163.85,
 'priceHint': 2,
 'currentTradingPeriod': {'pre': {'timezone': 'EST',
   'start': 1613120400,
   'end': 1613140200,
   'gmtoffset': -18000},
  'regular': {'timezone': 'EST',
   'start': 1613140200,
   'end': 1613163600,
   'gmtoffset': -18000},
  'post': {'timezone': 'EST',
   'start': 1613163600,
   'end': 1613178000,
   'gmtoffset': -18000}},
 'dataGranularity': '1d',
 'range': '10y',
 'validRanges': ['1d',
  '5d',
  '1mo',
  '3mo',
  '6mo',
  '1y',
  '2y',
  '5y',
  '10y',
  'ytd',
  'max']}

In [9]:
price = metadataAndPrice["price"]
price

,timestamp,low,high,open,close,volume
0,1297693800,162.850006,164.380005,164.179993,163.220001,4129800
1,1297780200,162.520004,163.570007,162.889999,162.839996,3768700
2,1297866600,162.750000,163.600006,163.330002,163.399994,3216000
3,1297953000,162.850006,164.669998,163.300003,164.240005,3230500
4,1298039400,164.100006,164.839996,164.460007,164.839996,4245000
...,...,...,...,...,...,...
2512,1612794600,122.400002,123.980003,122.620003,123.610001,5888000
2513,1612881000,121.050003,122.599998,121.900002,122.099998,4717000
2514,1612967400,121.209999,123.410004,123.029999,122.239998,4866800
2515,1613053800,120.629997,122.209999,122.000000,120.910004,5380600


In [10]:
basicStats = basic_stats(symbol)
basicStats

{'Previous Close': '120.91',
 'Open': '121.00',
 'Bid': '120.86 x 1000',
 'Ask': '120.91 x 800',
 "Day's Range": '120.09 - 121.36',
 '52 Week Range': '90.56 - 151.89',
 'Volume': '3,879,636',
 'Avg. Volume': '6,234,309',
 'Market Cap': '107.64B',
 'Beta (5Y Monthly)': '1.25',
 'PE Ratio (TTM)': '19.39',
 'EPS (TTM)': '6.23',
 'Earnings Date': 'Apr 19, 2021 - Apr 23, 2021',
 'Forward Dividend & Yield': '6.52 (5.40%)',
 'Ex-Dividend Date': 'Feb 09, 2021'}

In [11]:
extStats = extended_stock_stats(symbol)
extStats

{'Market Cap (intraday) 5': ['107.64B',
  '112.17B',
  '108.41B',
  '107.56B',
  '98.49B',
  '118.91B'],
 'Enterprise Value 3': ['160.30B',
  '166.96B',
  '164.08B',
  '165.14B',
  '157.78B',
  '179.58B'],
 'Trailing P/E ': ['19.71', '14.26', '13.79', '11.96', '10.49', '15.57'],
 'Forward P/E 1': ['10.95', '10.75', '9.98', '11.15', '8.31', '9.92'],
 'PEG Ratio (5 yr expected) 1': ['1.25',
  '9.35',
  '7.73',
  '9.69',
  '3.59',
  '22.56'],
 'Price/Sales (ttm)': ['1.47', '1.50', '1.44', '1.41', '1.28', '1.56'],
 'Price/Book (mrq)': ['5.23', '5.29', '5.28', '5.38', '4.73', '6.62'],
 'Enterprise Value/Revenue 3': ['2.18',
  '8.20',
  '9.34',
  '9.11',
  '8.98',
  '8.25'],
 'Enterprise Value/EBITDA 6': ['12.70',
  '50.52',
  '42.81',
  '46.22',
  '82.57',
  '29.95'],
 'Beta (5Y Monthly) ': '1.25',
 '52-Week Change 3': '-20.05%',
 'S&P500 52-Week Change 3': '16.75%',
 '52 Week High 3': '151.89',
 '52 Week Low 3': '90.56',
 '50-Day Moving Average 3': '124.29',
 '200-Day Moving Average 3': '1